In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Flatten, TimeDistributed

def create_model():
    # Model parameters
    n_joints = 17
    n_dimensions = 3
    n_frames = 5
    n_features = n_joints * n_dimensions  # 51 features = 17 joints * 3 dimensions

    # Model configuration
    model = Sequential([
        TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'), input_shape=(n_frames, n_features, 1)),
        TimeDistributed(Flatten()),
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(n_features)
    ])

    model.compile(optimizer='adam', loss='mse')
    return model

model = create_model()

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 5, 51, 64)        256       
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 5, 3264)          0         
 tributed)                                                       
                                                                 
 lstm_6 (LSTM)               (None, 5, 50)             663000    
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_3 (Dense)             (None, 51)                2601      
                                                                 
Total params: 686,057
Trainable params: 686,057
Non-tr

In [36]:
import numpy as np
import json

with open('sequence_data1.json', 'r') as file:
    data = json.load(file)
    sequences = np.array(data['sequences'])

    x_train = []
    y_train = []

    window_size = 5  # 5개의 이전 프레임을 사용하여 다음 프레임을 예측

    # 데이터에서 시퀀스 추출
    for sequence in sequences:
        # 각 시퀀스 내에서 가능한 모든 윈도우 추출
        for i in range(len(sequence) - window_size):
            x_train.append(sequence[i:i+window_size])
            y_train.append(sequence[i+window_size])

    # numpy 배열로 변환
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    # 데이터 형태 출력
    print("x_train shape:", x_train.shape)
    print("y_train shape:", y_train.shape)

x_train shape: (990, 5, 51)
y_train shape: (990, 51)


In [37]:
# X_train의 형태: (샘플 수, n_frames, n_features)
# y_train의 형태: (샘플 수, n_features) - 다음 프레임의 조인트 좌표

# 모델 트레이닝
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
31/31 [==============================] - 44s 1s/step - loss: 0.0680
Epoch 2/10
31/31 [==============================] - 8s 170ms/step - loss: 0.0104
Epoch 3/10
31/31 [==============================] - 36s 1s/step - loss: 0.0075
Epoch 4/10
31/31 [==============================] - 7s 231ms/step - loss: 0.0057
Epoch 5/10
31/31 [==============================] - 2s 46ms/step - loss: 0.0047
Epoch 6/10
31/31 [==============================] - 1s 45ms/step - loss: 0.0041
Epoch 7/10
31/31 [==============================] - 1s 47ms/step - loss: 0.0038
Epoch 8/10
31/31 [==============================] - 1s 47ms/step - loss: 0.0036
Epoch 9/10
31/31 [==============================] - 1s 47ms/step - loss: 0.0035
Epoch 10/10
31/31 [==============================] - 2s 49ms/step - loss: 0.0035


In [45]:
with open('unlabeled_data5.json', 'r') as file:
    data = json.load(file)
    sequences = np.array(data['sequences'])

    test_x_train = []
    test_y_train = []

    window_size = 5  # 5개의 이전 프레임을 사용하여 다음 프레임을 예측

    # 데이터에서 시퀀스 추출
    for sequence in sequences:
        # 각 시퀀스 내에서 가능한 모든 윈도우 추출
        for i in range(len(sequence) - window_size):
            test_x_train.append(sequence[i:i+window_size])
            test_y_train.append(sequence[i+window_size])

    # numpy 배열로 변환
    test_x_train = np.array(test_x_train)
    test_y_train = np.array(test_y_train)

    # 데이터 형태 출력
    print("test_x_train shape:", test_x_train.shape)
    print("test_y_train shape:", test_y_train.shape)

test_x_train shape: (55, 5, 51)
test_y_train shape: (55, 51)


In [46]:
# 예측 수행
predictions = model.predict(test_x_train)
def detect_anomalies(real, predicted, threshold=0.33):
    # 실제 값과 예측 값의 차이를 계산
    diff = np.linalg.norm(real - predicted, axis=1)
    anomalies = diff > threshold
    return anomalies, diff

# 이상 감지 실행
anomalies, differences = detect_anomalies(test_y_train, predictions)

print(anomalies)
print(differences)

idx = 6
for i in anomalies:
    if i == False:
        print("Frame",idx,"is False")
    idx += 1

2/2 [==============================] - 3s 3s/step
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]
[0.37613125 0.43522897 0.44057623 0.43521562 0.45311745 0.45529923
 0.44265627 0.47525426 0.38921494 0.46238241 0.44301224 0.44727659
 0.40474864 0.47833367 0.46996879 0.40931416 0.43936759 0.4303018
 0.45935298 0.60460224 0.50290843 0.62819098 0.75867157 0.65573009
 0.55941485 0.68911034 0.53619786 0.63178233 0.60895123 0.59032367
 0.52453737 0.56043372 0.63336731 0.76419413 0.59737777 0.55167095
 0.58585656 0.6435022  0.72205825 0.79800897 0.61881634 0.72507139
 0.70300933 0.51143496 0.55392677 0.5540556  0.70513693 0.60587784
 0.61770691 1.36767527 1.2427318  1.08964739 0.82887822 0.8522678
 0.95892351]


In [48]:
model.save('CNN_LSTM_V1.h5')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 5, 51, 64)        256       
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 5, 3264)          0         
 tributed)                                                       
                                                                 
 lstm_6 (LSTM)               (None, 5, 50)             663000    
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_3 (Dense)             (None, 51)                2601      
                                                                 
Total params: 686,057
Trainable params: 686,057
Non-tr